In [1]:
import os
import dotenv

dotenv.load_dotenv()

from datafin.aws import SecretsClient
from datafin.apis import PolygonClient


env_personal_aws_key = os.getenv('PERSONAL_AWS_ACCESS_KEY')
env_personal_aws_secret_key = os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY')

secrets = SecretsClient(
    aws_access_key=env_personal_aws_key,
    aws_secret_access_key=env_personal_aws_secret_key
)

In [2]:
pg = PolygonClient(secrets.get_polygon_api_key())

In [10]:
today = pg.get_aggs(
    'spy',
    1,
    'minute',
    '2020-06-11',
    '2025-06-10'
)

KeyboardInterrupt: 

In [ ]:
from polygon import RESTClient

client = RESTClient(secrets.get_polygon_api_key())

aggs = []
for a in client.list_aggs(
    "SPY",
    1,
    "minute",
    "2020-06-11",
    "2025-06-10",
    adjusted="true",
    sort="asc",
    limit=500000,
):
    aggs.append(a)

In [9]:
today

[Agg(open=589.39, high=613.23, low=546.87, close=559.39, volume=3353423934.0, vwap=583.8242, timestamp=1735707600000, transactions=33557592, otc=None),
 Agg(open=557.45, high=603.47, low=481.8, close=603.08, volume=4096618975.0, vwap=553.5851, timestamp=1743480000000, transactions=41062330, otc=None)]